In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import label, binary_dilation, binary_erosion, convolve
from itertools import product

class ARCSolutionGenerator:
    def __init__(self):
        self.pattern_handlers = [
            # Core handlers
            self.handle_empty_input,
            self.handle_identity,
            
            # 1D Raster transformations (NEW)
            self.handle_1d_raster_transform,
            self.handle_1d_pattern_repeat,
            self.handle_1d_sort_colors,
            self.handle_1d_run_length_decode,
            
            # Geometric transformations
            self.handle_rotate_90,
            self.handle_rotate_180,
            self.handle_rotate_270,
            self.handle_flip_horizontal,
            self.handle_flip_vertical,
            self.handle_transpose,
            self.handle_rotate_45,
            
            # Size transformations
            self.handle_upscale,
            self.handle_downscale,
            self.handle_pad_to_square,
            self.handle_crop_to_content,
            self.handle_zoom_center,
            
            # Color operations
            self.handle_color_mapping,
            self.handle_color_count,
            self.handle_color_frequency_sort,
            self.handle_replace_most_common,
            self.handle_color_propagation,
            self.handle_flood_fill,
            
            # Pattern operations
            self.handle_repeat_pattern,
            self.handle_tile_pattern,
            self.handle_mirror_pattern,
            self.handle_extract_pattern,
            self.handle_apply_mask,
            
            # Object operations
            self.handle_count_objects,
            self.handle_largest_object,
            self.handle_move_objects,
            self.handle_gravity,
            self.handle_connect_same_color,
            self.handle_outline_objects,
            
            # Grid operations
            self.handle_split_grid,
            self.handle_merge_grids,
            self.handle_overlay_grids,
            self.handle_subtract_grids,
            
            # Line operations
            self.handle_draw_lines,
            self.handle_detect_lines,
            self.handle_extend_lines,
            
            # Symmetry operations
            self.handle_make_symmetric,
            self.handle_complete_pattern,
            self.handle_find_symmetry_axis,
            
            # Advanced operations
            self.handle_conway_step,
            self.handle_cellular_automaton,
            self.handle_maze_solve,
            self.handle_path_finding,
            
            # Composite operations
            self.handle_multi_step_transform,
            self.handle_conditional_transform,
            self.handle_recursive_transform,
        ]
    
    # =============== 1D RASTER TRANSFORMATIONS ===============
    
    def to_1d_raster(self, grid, order='row_major'):
        """Convert 2D grid to 1D array with various ordering schemes"""
        h, w = len(grid), len(grid[0]) if grid else 0
        
        if order == 'row_major':  # Left to right, top to bottom
            return [grid[i][j] for i in range(h) for j in range(w)]
        elif order == 'col_major':  # Top to bottom, left to right
            return [grid[i][j] for j in range(w) for i in range(h)]
        elif order == 'row_major_rev':  # Right to left, bottom to top
            return [grid[i][j] for i in range(h-1,-1,-1) for j in range(w-1,-1,-1)]
        elif order == 'spiral':  # Spiral from outside to center
            result = []
            top, bottom, left, right = 0, h-1, 0, w-1
            while top <= bottom and left <= right:
                for j in range(left, right+1):
                    result.append(grid[top][j])
                top += 1
                for i in range(top, bottom+1):
                    result.append(grid[i][right])
                right -= 1
                if top <= bottom:
                    for j in range(right, left-1, -1):
                        result.append(grid[bottom][j])
                    bottom -= 1
                if left <= right:
                    for i in range(bottom, top-1, -1):
                        result.append(grid[i][left])
                    left += 1
            return result
        elif order == 'zigzag':  # Zigzag pattern
            result = []
            for i in range(h):
                if i % 2 == 0:
                    result.extend(grid[i])
                else:
                    result.extend(grid[i][::-1])
            return result
        return []
    
    def from_1d_raster(self, arr, h, w, order='row_major'):
        """Convert 1D array back to 2D grid"""
        if len(arr) != h * w:
            return None
            
        grid = [[0]*w for _ in range(h)]
        
        if order == 'row_major':
            for idx, val in enumerate(arr):
                i, j = idx // w, idx % w
                grid[i][j] = val
        elif order == 'col_major':
            for idx, val in enumerate(arr):
                j, i = idx // h, idx % h
                grid[i][j] = val
        elif order == 'row_major_rev':
            for idx, val in enumerate(arr):
                i, j = (h-1) - idx // w, (w-1) - idx % w
                grid[i][j] = val
        elif order == 'spiral':
            idx = 0
            top, bottom, left, right = 0, h-1, 0, w-1
            while top <= bottom and left <= right and idx < len(arr):
                for j in range(left, right+1):
                    if idx < len(arr):
                        grid[top][j] = arr[idx]
                        idx += 1
                top += 1
                for i in range(top, bottom+1):
                    if idx < len(arr):
                        grid[i][right] = arr[idx]
                        idx += 1
                right -= 1
                if top <= bottom:
                    for j in range(right, left-1, -1):
                        if idx < len(arr):
                            grid[bottom][j] = arr[idx]
                            idx += 1
                    bottom -= 1
                if left <= right:
                    for i in range(bottom, top-1, -1):
                        if idx < len(arr):
                            grid[i][left] = arr[idx]
                            idx += 1
                    left += 1
        elif order == 'zigzag':
            idx = 0
            for i in range(h):
                if i % 2 == 0:
                    for j in range(w):
                        grid[i][j] = arr[idx]
                        idx += 1
                else:
                    for j in range(w-1, -1, -1):
                        grid[i][j] = arr[idx]
                        idx += 1
        
        return grid
    
    def handle_1d_raster_transform(self, task_data, analysis):
        """Apply transformations in 1D space"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        h, w = inp.shape
        
        # Try different raster orders
        for order in ['row_major', 'col_major', 'zigzag', 'spiral']:
            arr_in = self.to_1d_raster(inp.tolist(), order)
            arr_out = self.to_1d_raster(out.tolist(), order)
            
            # Check for simple transformations
            # Reverse
            if arr_out == arr_in[::-1]:
                return f"""def p(g):
 h,w=len(g),len(g[0])
 arr={''.join([
  f"[g[i][j] for i in range(h) for j in range(w)]" if order=='row_major' else
  f"[g[i][j] for j in range(w) for i in range(h)]" if order=='col_major' else
  f"[g[i][j] for i in range(h-1,-1,-1) for j in range(w-1,-1,-1)]" if order=='row_major_rev' else
  "[]"
 ])}
 arr=arr[::-1]
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  {"i,j=idx//w,idx%w;res[i][j]=val" if order=='row_major' else
   "j,i=idx//h,idx%h;res[i][j]=val" if order=='col_major' else
   ""}
 return res
"""
            
            # Sort
            if sorted(arr_out) == sorted(arr_in) and arr_out == sorted(arr_in):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 arr=sorted([g[i][j] for i in range(h) for j in range(w)])
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr):
  i,j=idx//w,idx%w
  res[i][j]=val
 return res
"""
        return None
    
    def handle_1d_pattern_repeat(self, task_data, analysis):
        """Detect repeating patterns in 1D representation"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        arr_in = self.to_1d_raster(inp.tolist(), 'row_major')
        arr_out = self.to_1d_raster(out.tolist(), 'row_major')
        
        # Check if output is input repeated
        for repeat in [2, 3, 4]:
            if arr_out == arr_in * repeat:
                h_out, w_out = out.shape
                return f"""def p(g):
 arr=[g[i][j] for i in range(len(g)) for j in range(len(g[0]))]
 arr=arr*{repeat}
 h,w={h_out},{w_out}
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(arr[:h*w]):
  res[idx//w][idx%w]=val
 return res
"""
        return None
    
    def handle_1d_sort_colors(self, task_data, analysis):
        """Sort colors in 1D representation"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Group consecutive same colors
        arr_in = self.to_1d_raster(inp.tolist(), 'row_major')
        arr_out = self.to_1d_raster(out.tolist(), 'row_major')
        
        # Check if output groups same colors together
        color_groups = defaultdict(list)
        for i, c in enumerate(arr_in):
            color_groups[c].append(i)
        
        sorted_arr = []
        for color in sorted(color_groups.keys()):
            sorted_arr.extend([color] * len(color_groups[color]))
        
        if arr_out == sorted_arr:
            return """def p(g):
 h,w=len(g),len(g[0])
 arr=[g[i][j] for i in range(h) for j in range(w)]
 counts={}
 for c in arr:
  counts[c]=counts.get(c,0)+1
 res=[]
 for c in sorted(counts.keys()):
  res.extend([c]*counts[c])
 out=[[0]*w for _ in range(h)]
 for i,v in enumerate(res):
  out[i//w][i%w]=v
 return out
"""
        return None
    
    # =============== CORE HANDLERS ===============
    
    def handle_empty_input(self, task_data, analysis):
        """Handle empty input grids"""
        inp = np.array(task_data['train'][0]['input'])
        out = np.array(task_data['train'][0]['output'])
        
        if inp.size == 0 or inp.shape[0] == 0:
            return f"""def p(g):
 return {out.tolist()}
"""
        return None
    
    def handle_identity(self, task_data, analysis):
        """Check if output equals input"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if np.array_equal(inp, out):
            return """def p(g):
 return g
"""
        return None
    
    # =============== GEOMETRIC TRANSFORMATIONS ===============
    
    def handle_transpose(self, task_data, analysis):
        """Transpose the grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if np.array_equal(inp.T, out):
            return """def p(g):
 return[list(r)for r in zip(*g)]
"""
        return None
    
    def handle_rotate_45(self, task_data, analysis):
        """Rotate 45 degrees (diagonal transform)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        h, w = inp.shape
        
        # Create diagonal representation
        diagonals = defaultdict(list)
        for i in range(h):
            for j in range(w):
                diagonals[i+j].append(inp[i,j])
        
        # Check if output matches diagonal pattern
        if len(diagonals) == out.shape[0]:
            out_rows = []
            for d in sorted(diagonals.keys()):
                out_rows.append(diagonals[d])
            
            # Pad to make rectangular
            max_len = max(len(row) for row in out_rows)
            for row in out_rows:
                row.extend([0] * (max_len - len(row)))
            
            if np.array_equal(np.array(out_rows), out):
                return """def p(g):
 h,w=len(g),len(g[0])
 d={}
 for i in range(h):
  for j in range(w):
   k=i+j
   if k not in d:d[k]=[]
   d[k].append(g[i][j])
 res=[]
 for k in sorted(d.keys()):
  res.append(d[k])
 m=max(len(r)for r in res)
 for r in res:
  r.extend([0]*(m-len(r)))
 return res
"""
        return None
    
    # =============== SIZE TRANSFORMATIONS ===============
    
    def handle_pad_to_square(self, task_data, analysis):
        """Pad to make square grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        h, w = inp.shape
        
        if h != w and out.shape[0] == out.shape[1]:
            size = max(h, w)
            if out.shape == (size, size):
                # Check if input is centered in output
                pad_h = (size - h) // 2
                pad_w = (size - w) // 2
                
                test = np.zeros((size, size), dtype=int)
                test[pad_h:pad_h+h, pad_w:pad_w+w] = inp
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 s=max(h,w)
 ph,pw=(s-h)//2,(s-w)//2
 res=[[0]*s for _ in range(s)]
 for i in range(h):
  for j in range(w):
   res[ph+i][pw+j]=g[i][j]
 return res
"""
        return None
    
    def handle_crop_to_content(self, task_data, analysis):
        """Crop to non-zero content"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find bounding box of non-zero elements
        rows, cols = np.where(inp > 0)
        if len(rows) == 0:
            return None
            
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        
        cropped = inp[min_row:max_row+1, min_col:max_col+1]
        
        if np.array_equal(cropped, out):
            return """def p(g):
 rows,cols=[],[]
 for i in range(len(g)):
  for j in range(len(g[0])):
   if g[i][j]>0:
    rows.append(i)
    cols.append(j)
 if not rows:return g
 r0,r1=min(rows),max(rows)+1
 c0,c1=min(cols),max(cols)+1
 return[r[c0:c1]for r in g[r0:r1]]
"""
        return None
    
    def handle_zoom_center(self, task_data, analysis):
        """Zoom into center of grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        h, w = inp.shape
        oh, ow = out.shape
        
        if oh < h and ow < w:
            # Check if output is center crop
            y = (h - oh) // 2
            x = (w - ow) // 2
            
            if np.array_equal(inp[y:y+oh, x:x+ow], out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 oh,ow={oh},{ow}
 y,x=(h-oh)//2,(w-ow)//2
 return[r[x:x+ow]for r in g[y:y+oh]]
"""
        return None
    
    # =============== COLOR OPERATIONS ===============
    
    def handle_color_count(self, task_data, analysis):
        """Replace with color count"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Count unique colors
        unique_colors = len(np.unique(inp))
        
        if np.all(out == unique_colors):
            return f"""def p(g):
 colors=set()
 for r in g:
  for c in r:
   colors.add(c)
 n=len(colors)
 return[[n]*len(g[0])for _ in g]
"""
        return None
    
    def handle_color_frequency_sort(self, task_data, analysis):
        """Sort by color frequency"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Get color frequencies
        flat = inp.flatten()
        counts = Counter(flat)
        
        # Create frequency-based mapping
        sorted_colors = sorted(counts.keys(), key=lambda x: (-counts[x], x))
        color_map = {c: i+1 for i, c in enumerate(sorted_colors) if c != 0}
        color_map[0] = 0
        
        # Apply mapping
        test = np.vectorize(lambda x: color_map.get(x, x))(inp)
        
        if np.array_equal(test, out):
            map_str = ','.join(f'{k}:{v}' for k,v in color_map.items())
            return f"""def p(g):
 from collections import Counter
 c=Counter(x for r in g for x in r)
 s=sorted(c.keys(),key=lambda x:(-c[x],x))
 m={{c:i+1 for i,c in enumerate(s)if c!=0}}
 m[0]=0
 return[[m.get(x,x)for x in r]for r in g]
"""
        return None
    
    def handle_replace_most_common(self, task_data, analysis):
        """Replace most common color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find most common color
        counts = Counter(inp.flatten())
        most_common = counts.most_common(1)[0][0]
        
        # Try different replacements
        for new_color in range(10):
            test = np.where(inp == most_common, new_color, inp)
            if np.array_equal(test, out):
                return f"""def p(g):
 from collections import Counter
 c=Counter(x for r in g for x in r)
 mc=c.most_common(1)[0][0]
 return[[{new_color} if x==mc else x for x in r]for r in g]
"""
        return None
    
    def handle_color_propagation(self, task_data, analysis):
        """Propagate colors in directions"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try propagating non-zero colors
        h, w = inp.shape
        
        # Horizontal propagation
        test = np.copy(inp)
        for i in range(h):
            last_color = 0
            for j in range(w):
                if inp[i,j] != 0:
                    last_color = inp[i,j]
                elif last_color != 0:
                    test[i,j] = last_color
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(len(g)):
  lc=0
  for j in range(len(g[0])):
   if g[i][j]!=0:
    lc=g[i][j]
   elif lc!=0:
    res[i][j]=lc
 return res
"""
        
        # Vertical propagation
        test = np.copy(inp)
        for j in range(w):
            last_color = 0
            for i in range(h):
                if inp[i,j] != 0:
                    last_color = inp[i,j]
                elif last_color != 0:
                    test[i,j] = last_color
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for j in range(len(g[0])):
  lc=0
  for i in range(len(g)):
   if g[i][j]!=0:
    lc=g[i][j]
   elif lc!=0:
    res[i][j]=lc
 return res
"""
        return None
    
    def handle_flood_fill(self, task_data, analysis):
        """Flood fill from corners or edges"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Simple flood fill from (0,0)
        if inp[0,0] == 0 and out[0,0] != 0:
            fill_color = out[0,0]
            
            # Do flood fill
            h, w = inp.shape
            filled = np.copy(inp)
            to_fill = [(0, 0)]
            filled[0,0] = fill_color
            
            while to_fill:
                y, x = to_fill.pop()
                for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < h and 0 <= nx < w and filled[ny,nx] == 0:
                        filled[ny,nx] = fill_color
                        to_fill.append((ny, nx))
            
            if np.array_equal(filled, out):
                return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 fc={fill_color}
 q=[(0,0)]
 res[0][0]=fc
 while q:
  y,x=q.pop()
  for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
   ny,nx=y+dy,x+dx
   if 0<=ny<h and 0<=nx<w and res[ny][nx]==0:
    res[ny][nx]=fc
    q.append((ny,nx))
 return res
"""
        return None
    
    # =============== PATTERN OPERATIONS ===============
    
    def handle_tile_pattern(self, task_data, analysis):
        """Tile a small pattern"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try different tile sizes
        ih, iw = inp.shape
        oh, ow = out.shape
        
        for th in range(1, min(ih+1, 5)):
            for tw in range(1, min(iw+1, 5)):
                if oh % th == 0 and ow % tw == 0:
                    tile = inp[:th, :tw]
                    tiled = np.tile(tile, (oh//th, ow//tw))
                    
                    if np.array_equal(tiled, out):
                        return f"""def p(g):
 tile=[r[:{tw}]for r in g[:{th}]]
 h,w={oh},{ow}
 res=[]
 for i in range(h):
  row=[]
  for j in range(w):
   row.append(tile[i%{th}][j%{tw}])
  res.append(row)
 return res
"""
        return None
    
    def handle_mirror_pattern(self, task_data, analysis):
        """Create mirrored patterns"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Horizontal mirror
        mirrored = np.hstack([inp, np.fliplr(inp)])
        if np.array_equal(mirrored, out):
            return """def p(g):
 return[r+r[::-1]for r in g]
"""
        
        # Vertical mirror
        mirrored = np.vstack([inp, np.flipud(inp)])
        if np.array_equal(mirrored, out):
            return """def p(g):
 return g+g[::-1]
"""
        
        # 4-way mirror
        top = np.hstack([inp, np.fliplr(inp)])
        bottom = np.hstack([np.flipud(inp), np.flipud(np.fliplr(inp))])
        mirrored = np.vstack([top, bottom])
        if np.array_equal(mirrored, out):
            return """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b
"""
        return None
    
    def handle_extract_pattern(self, task_data, analysis):
        """Extract repeating pattern"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is a subpattern of input
        ih, iw = inp.shape
        oh, ow = out.shape
        
        if oh <= ih and ow <= iw:
            # Try different positions
            for y in range(ih - oh + 1):
                for x in range(iw - ow + 1):
                    if np.array_equal(inp[y:y+oh, x:x+ow], out):
                        # Check if this position has special property
                        # E.g., top-left corner
                        if y == 0 and x == 0:
                            return f"""def p(g):
 return[r[:{ow}]for r in g[:{oh}]]
"""
                        # Center
                        if y == (ih-oh)//2 and x == (iw-ow)//2:
                            return f"""def p(g):
 h,w=len(g),len(g[0])
 oh,ow={oh},{ow}
 y,x=(h-oh)//2,(w-ow)//2
 return[r[x:x+ow]for r in g[y:y+oh]]
"""
        return None
    
    def handle_apply_mask(self, task_data, analysis):
        """Apply mask based on pattern"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Check if output is masked version of input
        # E.g., keep only certain positions
        mask = (out != 0)
        
        # Check for checkerboard mask
        checkerboard = np.indices(inp.shape).sum(axis=0) % 2
        if np.array_equal(mask, checkerboard):
            test = np.where(checkerboard, inp, 0)
            if np.array_equal(test, out):
                return """def p(g):
 return[[g[i][j]if(i+j)%2 else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
        
        # Check for diagonal mask
        diag_mask = np.eye(inp.shape[0], inp.shape[1], dtype=bool)
        if np.array_equal(mask, diag_mask):
            test = np.where(diag_mask, inp, 0)
            if np.array_equal(test, out):
                return """def p(g):
 return[[g[i][j]if i==j else 0 for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None
    
    # =============== OBJECT OPERATIONS ===============
    
    def handle_count_objects(self, task_data, analysis):
        """Count connected components"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        labeled, n = label(inp > 0)
        
        # Output is count
        if out.shape == (1,1) and out[0,0] == n:
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]]
"""
        
        # Output shows count everywhere
        if np.all(out == n):
            return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 _,n=label(np.array(g)>0)
 return[[n]*len(g[0])for _ in g]
"""
        return None
    
    def handle_largest_object(self, task_data, analysis):
        """Keep only largest object"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        labeled, n = label(inp > 0)
        if n == 0:
            return None
        
        # Find largest
        sizes = [(labeled == i).sum() for i in range(1, n+1)]
        largest = np.argmax(sizes) + 1
        
        # Keep only largest
        result = np.where(labeled == largest, inp, 0)
        
        if np.array_equal(result, out):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr=np.array(g)
 l,n=label(arr>0)
 if n==0:return g
 sizes=[(l==i).sum()for i in range(1,n+1)]
 largest=np.argmax(sizes)+1
 return np.where(l==largest,arr,0).tolist()
"""
        return None
    
    def handle_move_objects(self, task_data, analysis):
        """Move objects by fixed offset"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Find possible translations
        for dy in range(-3, 4):
            for dx in range(-3, 4):
                if dy == 0 and dx == 0:
                    continue
                
                # Try moving all non-zero pixels
                test = np.zeros_like(inp)
                h, w = inp.shape
                
                for i in range(h):
                    for j in range(w):
                        if inp[i,j] != 0:
                            ni, nj = i + dy, j + dx
                            if 0 <= ni < h and 0 <= nj < w:
                                test[ni,nj] = inp[i,j]
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    ni,nj=i+{dy},j+{dx}
    if 0<=ni<h and 0<=nj<w:
     res[ni][nj]=g[i][j]
 return res
"""
        return None
    
    def handle_gravity(self, task_data, analysis):
        """Apply gravity to objects"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Down gravity
        test = np.zeros_like(inp)
        for j in range(w):
            col = inp[:,j]
            non_zero = col[col != 0]
            test[h-len(non_zero):,j] = non_zero
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[h-len(nz)+k][j]=v
 return res
"""
        
        # Up gravity
        test = np.zeros_like(inp)
        for j in range(w):
            col = inp[:,j]
            non_zero = col[col != 0]
            test[:len(non_zero),j] = non_zero
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for j in range(w):
  nz=[g[i][j]for i in range(h)if g[i][j]!=0]
  for k,v in enumerate(nz):
   res[k][j]=v
 return res
"""
        return None
    
    def handle_connect_same_color(self, task_data, analysis):
        """Connect pixels of same color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # For each color, connect all pixels
        test = np.copy(inp)
        
        for color in range(1, 10):
            positions = np.argwhere(inp == color)
            if len(positions) < 2:
                continue
            
            # Connect all pairs
            for i in range(len(positions)):
                for j in range(i+1, len(positions)):
                    y1, x1 = positions[i]
                    y2, x2 = positions[j]
                    
                    # Draw line between points
                    if y1 == y2:  # Horizontal line
                        for x in range(min(x1,x2), max(x1,x2)+1):
                            test[y1,x] = color
                    elif x1 == x2:  # Vertical line
                        for y in range(min(y1,y2), max(y1,y2)+1):
                            test[y,x1] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for c in range(1,10):
  pos=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]==c]
  for i in range(len(pos)):
   for j in range(i+1,len(pos)):
    y1,x1=pos[i]
    y2,x2=pos[j]
    if y1==y2:
     for x in range(min(x1,x2),max(x1,x2)+1):
      res[y1][x]=c
    elif x1==x2:
     for y in range(min(y1,y2),max(y1,y2)+1):
      res[y][x1]=c
 return res
"""
        return None
    
    def handle_outline_objects(self, task_data, analysis):
        """Keep only object outlines"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Find edges
        h, w = inp.shape
        edges = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    # Check if on edge
                    is_edge = False
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if not (0 <= ni < h and 0 <= nj < w) or inp[ni,nj] == 0:
                            is_edge = True
                            break
                    if is_edge:
                        edges[i,j] = inp[i,j]
        
        if np.array_equal(edges, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    edge=False
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if not(0<=ni<h and 0<=nj<w)or g[ni][nj]==0:
      edge=True
      break
    if edge:
     res[i][j]=g[i][j]
 return res
"""
        return None
    
    # =============== LINE OPERATIONS ===============
    
    def handle_draw_lines(self, task_data, analysis):
        """Draw lines through grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Check for cross pattern
        ch, cw = h // 2, w // 2
        test = np.copy(inp)
        
        # Horizontal line through center
        if h > 0:
            test[ch,:] = 1
        # Vertical line through center  
        if w > 0:
            test[:,cw] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 if h>0:
  res[h//2]=[1]*w
 if w>0:
  for i in range(h):
   res[i][w//2]=1
 return res
"""
        
        # Check for diagonal lines
        test = np.copy(inp)
        for i in range(min(h,w)):
            test[i,i] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 for i in range(min(len(g),len(g[0]))):
  res[i][i]=1
 return res
"""
        return None
    
    def handle_detect_lines(self, task_data, analysis):
        """Detect and highlight lines"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        # Detect horizontal lines
        for i in range(h):
            if np.all(inp[i,:] == inp[i,0]) and inp[i,0] != 0:
                test[i,:] = inp[i,0]
        
        # Detect vertical lines
        for j in range(w):
            if np.all(inp[:,j] == inp[0,j]) and inp[0,j] != 0:
                test[:,j] = inp[0,j]
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  if all(g[i][j]==g[i][0]for j in range(w))and g[i][0]!=0:
   res[i]=[g[i][0]]*w
 for j in range(w):
  if all(g[i][j]==g[0][j]for i in range(h))and g[0][j]!=0:
   for i in range(h):
    res[i][j]=g[0][j]
 return res
"""
        return None
    
    def handle_extend_lines(self, task_data, analysis):
        """Extend existing lines"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        test = np.copy(inp)
        
        # Extend horizontal segments
        for i in range(h):
            segments = []
            start = None
            
            for j in range(w):
                if inp[i,j] != 0:
                    if start is None:
                        start = j
                else:
                    if start is not None:
                        segments.append((start, j-1, inp[i,start]))
                        start = None
            
            if start is not None:
                segments.append((start, w-1, inp[i,start]))
            
            # Extend segments
            for s, e, color in segments:
                if e - s >= 1:  # Line of at least 2 pixels
                    test[i,:] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  segs=[]
  s=None
  for j in range(w):
   if g[i][j]!=0:
    if s is None:
     s=j
   else:
    if s is not None:
     segs.append((s,j-1,g[i][s]))
     s=None
  if s is not None:
   segs.append((s,w-1,g[i][s]))
  for s,e,c in segs:
   if e-s>=1:
    res[i]=[c]*w
 return res
"""
        return None
    
    # =============== SYMMETRY OPERATIONS ===============
    
    def handle_make_symmetric(self, task_data, analysis):
        """Make grid symmetric"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Horizontal symmetry (top half copied to bottom)
        test = np.copy(inp)
        for i in range(h//2):
            test[h-1-i,:] = test[i,:]
        
        if np.array_equal(test, out):
            return """def p(g):
 h=len(g)
 res=[r[:]for r in g]
 for i in range(h//2):
  res[h-1-i]=res[i][:]
 return res
"""
        
        # Vertical symmetry (left half copied to right)
        test = np.copy(inp)
        for j in range(w//2):
            test[:,w-1-j] = test[:,j]
        
        if np.array_equal(test, out):
            return """def p(g):
 w=len(g[0])
 res=[r[:]for r in g]
 for i in range(len(g)):
  for j in range(w//2):
   res[i][w-1-j]=res[i][j]
 return res
"""
        return None
    
    def handle_complete_pattern(self, task_data, analysis):
        """Complete partial pattern"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output completes a symmetric pattern
        if inp.shape == out.shape:
            # Try different symmetry completions
            h, w = inp.shape
            
            # Complete diagonal symmetry
            test = np.copy(inp)
            for i in range(h):
                for j in range(w):
                    if i < j and inp[i,j] != 0:
                        test[j,i] = inp[i,j]
                    elif i > j and inp[i,j] != 0:
                        test[j,i] = inp[i,j]
            
            if np.array_equal(test, out):
                return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 for i in range(h):
  for j in range(w):
   if i<j and g[i][j]!=0:
    res[j][i]=g[i][j]
   elif i>j and g[i][j]!=0:
    res[j][i]=g[i][j]
 return res
"""
        return None
    
    def handle_find_symmetry_axis(self, task_data, analysis):
        """Find and mark symmetry axis"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Check vertical symmetry and mark axis
        is_vsym = True
        for i in range(h):
            for j in range(w//2):
                if inp[i,j] != inp[i,w-1-j]:
                    is_vsym = False
                    break
        
        if is_vsym:
            test = np.copy(inp)
            test[:,w//2] = 9  # Mark center line
            if np.array_equal(test, out):
                return """def p(g):
 res=[r[:]for r in g]
 w=len(g[0])
 for i in range(len(g)):
  res[i][w//2]=9
 return res
"""
        return None
    
    # =============== ADVANCED OPERATIONS ===============
    
    def handle_conway_step(self, task_data, analysis):
        """Conway's Game of Life step"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                # Count neighbors
                neighbors = 0
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if di == 0 and dj == 0:
                            continue
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                            neighbors += 1
                
                # Apply rules
                if inp[i,j] != 0:
                    if neighbors in [2, 3]:
                        test[i,j] = inp[i,j]
                else:
                    if neighbors == 3:
                        test[i,j] = 1
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   n=0
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     if di==0 and dj==0:continue
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      n+=1
   if g[i][j]!=0:
    if n in[2,3]:
     res[i][j]=g[i][j]
   else:
    if n==3:
     res[i][j]=1
 return res
"""
        return None
    
    def handle_cellular_automaton(self, task_data, analysis):
        """General cellular automaton rules"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        h, w = inp.shape
        
        # Rule: Cell becomes color of majority of neighbors
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                # Count neighbor colors
                color_counts = defaultdict(int)
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w:
                            color_counts[inp[ni,nj]] += 1
                
                # Find most common color
                most_common = max(color_counts.items(), key=lambda x: x[1])[0]
                test[i,j] = most_common
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   cc={}
   for di in[-1,0,1]:
    for dj in[-1,0,1]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w:
      c=g[ni][nj]
      cc[c]=cc.get(c,0)+1
   mc=max(cc.items(),key=lambda x:x[1])[0]
   res[i][j]=mc
 return res
"""
        return None
    
    def handle_maze_solve(self, task_data, analysis):
        """Simple maze solving"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find start (color 2) and end (color 3)
        start = np.argwhere(inp == 2)
        end = np.argwhere(inp == 3)
        
        if len(start) == 1 and len(end) == 1:
            sy, sx = start[0]
            ey, ex = end[0]
            
            # Simple path finding (BFS)
            h, w = inp.shape
            visited = set()
            queue = [(sy, sx, [(sy, sx)])]
            path = None
            
            while queue and not path:
                y, x, current_path = queue.pop(0)
                
                if (y, x) == (ey, ex):
                    path = current_path
                    break
                
                if (y, x) in visited:
                    continue
                visited.add((y, x))
                
                for dy, dx in [(0,1), (1,0), (0,-1), (-1,0)]:
                    ny, nx = y + dy, x + dx
                    if (0 <= ny < h and 0 <= nx < w and 
                        inp[ny,nx] != 1 and (ny,nx) not in visited):
                        queue.append((ny, nx, current_path + [(ny, nx)]))
            
            if path:
                # Mark path in output
                test = np.copy(inp)
                for y, x in path[1:-1]:  # Exclude start and end
                    test[y,x] = 4
                
                if np.array_equal(test, out):
                    return """def p(g):
 import numpy as np
 h,w=len(g),len(g[0])
 inp=np.array(g)
 start=np.argwhere(inp==2)
 end=np.argwhere(inp==3)
 if len(start)==1 and len(end)==1:
  sy,sx=start[0]
  ey,ex=end[0]
  visited=set()
  queue=[(sy,sx,[(sy,sx)])]
  path=None
  while queue and not path:
   y,x,cp=queue.pop(0)
   if(y,x)==(ey,ex):
    path=cp
    break
   if(y,x)in visited:
    continue
   visited.add((y,x))
   for dy,dx in[(0,1),(1,0),(0,-1),(-1,0)]:
    ny,nx=y+dy,x+dx
    if 0<=ny<h and 0<=nx<w and g[ny][nx]!=1 and(ny,nx)not in visited:
     queue.append((ny,nx,cp+[(ny,nx)]))
  if path:
   res=[r[:]for r in g]
   for y,x in path[1:-1]:
    res[y][x]=4
   return res
 return g
"""
        return None
    
    def handle_path_finding(self, task_data, analysis):
        """Find shortest path between points"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find all colored points
        colors = defaultdict(list)
        h, w = inp.shape
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    colors[inp[i,j]].append((i,j))
        
        # Connect same colored points
        test = np.copy(inp)
        
        for color, points in colors.items():
            if len(points) == 2:
                # Draw line between two points
                (y1, x1), (y2, x2) = points
                
                # Simple Manhattan path
                if y1 == y2:
                    for x in range(min(x1,x2), max(x1,x2)+1):
                        test[y1,x] = color
                elif x1 == x2:
                    for y in range(min(y1,y2), max(y1,y2)+1):
                        test[y,x1] = color
                else:
                    # L-shaped path
                    for y in range(min(y1,y2), max(y1,y2)+1):
                        test[y,x1] = color
                    for x in range(min(x1,x2), max(x1,x2)+1):
                        test[y2,x] = color
        
        if np.array_equal(test, out):
            return """def p(g):
 from collections import defaultdict
 h,w=len(g),len(g[0])
 colors=defaultdict(list)
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    colors[g[i][j]].append((i,j))
 res=[r[:]for r in g]
 for c,pts in colors.items():
  if len(pts)==2:
   (y1,x1),(y2,x2)=pts
   if y1==y2:
    for x in range(min(x1,x2),max(x1,x2)+1):
     res[y1][x]=c
   elif x1==x2:
    for y in range(min(y1,y2),max(y1,y2)+1):
     res[y][x1]=c
   else:
    for y in range(min(y1,y2),max(y1,y2)+1):
     res[y][x1]=c
    for x in range(min(x1,x2),max(x1,x2)+1):
     res[y2][x]=c
 return res
"""
        return None
    
    # =============== GRID OPERATIONS ===============
    
    def handle_split_grid(self, task_data, analysis):
        """Split grid into regions"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        h, w = inp.shape
        oh, ow = out.shape
        
        # Check if output is one quadrant
        if oh == h//2 and ow == w//2:
            # Top-left
            if np.array_equal(inp[:h//2, :w//2], out):
                return """def p(g):
 h,w=len(g),len(g[0])
 return[r[:w//2]for r in g[:h//2]]
"""
            # Top-right
            if np.array_equal(inp[:h//2, w//2:], out):
                return """def p(g):
 h,w=len(g),len(g[0])
 return[r[w//2:]for r in g[:h//2]]
"""
        return None
    
    def handle_merge_grids(self, task_data, analysis):
        """Merge multiple grids"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is input repeated
        h, w = inp.shape
        oh, ow = out.shape
        
        # 2x2 tiling
        if oh == 2*h and ow == 2*w:
            test = np.vstack([
                np.hstack([inp, inp]),
                np.hstack([inp, inp])
            ])
            if np.array_equal(test, out):
                return """def p(g):
 return[r+r for r in g]+[r+r for r in g]
"""
        return None
    
    def handle_overlay_grids(self, task_data, analysis):
        """Overlay patterns"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Create simple overlay patterns
        h, w = inp.shape
        
        # Grid lines overlay
        test = np.copy(inp)
        # Add vertical lines every 3 pixels
        for j in range(0, w, 3):
            test[:,j] = np.where(test[:,j] == 0, 1, test[:,j])
        # Add horizontal lines every 3 pixels  
        for i in range(0, h, 3):
            test[i,:] = np.where(test[i,:] == 0, 1, test[i,:])
        
        if np.array_equal(test, out):
            return """def p(g):
 res=[r[:]for r in g]
 h,w=len(g),len(g[0])
 for j in range(0,w,3):
  for i in range(h):
   if res[i][j]==0:
    res[i][j]=1
 for i in range(0,h,3):
  for j in range(w):
   if res[i][j]==0:
    res[i][j]=1
 return res
"""
        return None
    
    def handle_subtract_grids(self, task_data, analysis):
        """Subtract one pattern from another"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Remove all of specific color
        for color in range(1, 10):
            test = np.where(inp == color, 0, inp)
            if np.array_equal(test, out):
                return f"""def p(g):
 return[[0 if x=={color} else x for x in r]for r in g]
"""
        
        # XOR operation (remove overlaps)
        # This would need another grid as input
        return None
    
    # =============== COMPOSITE OPERATIONS ===============
    
    def handle_multi_step_transform(self, task_data, analysis):
        """Apply multiple transformations"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try rotate then scale
        for rotation in [1, 2, 3]:
            rotated = np.rot90(inp, rotation)
            
            # Then try scale
            for scale in [2, 3]:
                if (rotated.shape[0] * scale == out.shape[0] and
                    rotated.shape[1] * scale == out.shape[1]):
                    
                    scaled = np.repeat(np.repeat(rotated, scale, axis=0), scale, axis=1)
                    
                    if np.array_equal(scaled, out):
                        return f"""def p(g):
 import numpy as np
 r=np.rot90(g,{rotation})
 return np.repeat(np.repeat(r,{scale},axis=0),{scale},axis=1).tolist()
"""
        return None
    
    def handle_conditional_transform(self, task_data, analysis):
        """Transform based on conditions"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Color based on neighbor count
        h, w = inp.shape
        test = np.zeros_like(inp)
        
        for i in range(h):
            for j in range(w):
                if inp[i,j] != 0:
                    # Count non-zero neighbors
                    count = 0
                    for di, dj in [(0,1), (1,0), (0,-1), (-1,0)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < h and 0 <= nj < w and inp[ni,nj] != 0:
                            count += 1
                    
                    # Map count to color
                    test[i,j] = count + 1
        
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   if g[i][j]!=0:
    c=0
    for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w and g[ni][nj]!=0:
      c+=1
    res[i][j]=c+1
 return res
"""
        return None
    
    def handle_recursive_transform(self, task_data, analysis):
        """Apply transformation recursively"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Simple recursive fill
        h, w = inp.shape
        
        # Fill enclosed areas recursively
        test = np.copy(inp)
        changed = True
        
        while changed:
            changed = False
            for i in range(1, h-1):
                for j in range(1, w-1):
                    if test[i,j] == 0:
                        # Check if surrounded
                        neighbors = [test[i+di,j+dj] for di,dj in [(0,1),(1,0),(0,-1),(-1,0)]]
                        
                        if all(n != 0 for n in neighbors):
                            # Get most common neighbor color
                            color_counts = Counter(neighbors)
                            test[i,j] = color_counts.most_common(1)[0][0]
                            changed = True
        
        if np.array_equal(test, out):
            return """def p(g):
 from collections import Counter
 h,w=len(g),len(g[0])
 res=[r[:]for r in g]
 changed=True
 while changed:
  changed=False
  for i in range(1,h-1):
   for j in range(1,w-1):
    if res[i][j]==0:
     n=[res[i+di][j+dj]for di,dj in[(0,1),(1,0),(0,-1),(-1,0)]]
     if all(x!=0 for x in n):
      c=Counter(n)
      res[i][j]=c.most_common(1)[0][0]
      changed=True
 return res
"""
        return None
    
    def handle_1d_run_length_decode(self, task_data, analysis):
        """Decode run-length encoded patterns"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if input encodes runs
        arr_in = self.to_1d_raster(inp.tolist(), 'row_major')
        
        # Simple run-length: alternating value and count
        decoded = []
        i = 0
        while i < len(arr_in) - 1:
            value = arr_in[i]
            count = arr_in[i+1]
            if count > 0 and count < 10:
                decoded.extend([value] * count)
            i += 2
        
        # Check if decoded matches output
        if len(decoded) > 0:
            oh, ow = out.shape
            if len(decoded) == oh * ow:
                test_grid = self.from_1d_raster(decoded, oh, ow, 'row_major')
                if test_grid and np.array_equal(np.array(test_grid), out):
                    return f"""def p(g):
 arr=[g[i][j]for i in range(len(g))for j in range(len(g[0]))]
 d=[]
 i=0
 while i<len(arr)-1:
  v,c=arr[i],arr[i+1]
  if c>0 and c<10:
   d.extend([v]*c)
  i+=2
 h,w={oh},{ow}
 res=[[0]*w for _ in range(h)]
 for idx,val in enumerate(d[:h*w]):
  res[idx//w][idx%w]=val
 return res
"""
        return None
    
    # Continue with more handlers...
    def handle_upscale(self, task_data, analysis):
        """Upscale by integer factor"""
        return self.handle_upscale_nearest(task_data, analysis)
    
    def handle_downscale(self, task_data, analysis):
        """Downscale by integer factor"""
        return self.handle_downscale_sample(task_data, analysis)
    
    # ----------------- Fixed Base Handlers -----------------
    
    def handle_upscale_nearest(self, task_data, analysis):
        """Upscale using nearest neighbor"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]*h_scale == out.shape[0] and 
                    inp.shape[1]*w_scale == out.shape[1]):
                    test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[[x for x in row for _ in range({w_scale})]
        for row in g for _ in range({h_scale})]
"""
        return None
    
    def handle_downscale_sample(self, task_data, analysis):
        """Downscale by sampling"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]//h_scale == out.shape[0] and 
                    inp.shape[1]//w_scale == out.shape[1]):
                    test = inp[::h_scale, ::w_scale]
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return[row[::{w_scale}]for row in g[::{h_scale}]]
"""
        return None
    
    def handle_rotate_90(self, task_data, analysis):
        """Rotate 90 degrees clockwise"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, -1), out):
            return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
        return None
    
    def handle_rotate_180(self, task_data, analysis):
        """Rotate 180 degrees"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, 2), out):
            return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
        return None
    
    def handle_rotate_270(self, task_data, analysis):
        """Rotate 270 degrees clockwise"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, 1), out):
            return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
        return None
    
    def handle_flip_horizontal(self, task_data, analysis):
        """Flip horizontally"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.fliplr(inp), out):
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None
    
    def handle_flip_vertical(self, task_data, analysis):
        """Flip vertically"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.flipud(inp), out):
            return """def p(g):
 return g[::-1]
"""
        return None
    
    def handle_repeat_pattern(self, task_data, analysis):
        """Repeat pattern to fill grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try different pattern sizes
        for h in range(1, min(5, inp.shape[0]+1)):
            for w in range(1, min(5, inp.shape[1]+1)):
                pattern = inp[:h,:w]
                test = np.tile(pattern, (out.shape[0]//h + 1, out.shape[1]//w + 1))
                test = test[:out.shape[0], :out.shape[1]]
                if np.array_equal(out, test):
                    return f"""def p(g):
 pat=[r[:{w}]for r in g[:{h}]]
 return[r[:len(g[0])]for _ in range(len(g))
  for r in pat*(len(g)//{h}+1)][:len(g)]
"""
        return None
    
    def handle_color_mapping(self, task_data, analysis):
        """Map colors based on analysis"""
        color_map = {}
        for pos, changes in analysis['color_changes'].items():
            if len(changes) == 1:
                src, dest = next(iter(changes))
                if src not in color_map:
                    color_map[src] = dest
                elif color_map[src] != dest:
                    return None
        
        if not color_map:
            return None
        
        cases = '\n'.join([f'  {int(s)}:{int(d)},' for s,d in color_map.items()])
        return f"""def p(g):
 m={{{cases}}}
 return[[m.get(x,x)for x in r]for r in g]
"""
    
    # ----------------- Utility Methods -----------------
    
    def analyze_task(self, task_data):
        """Analyze patterns in task"""
        all_examples = task_data['train'] + task_data['test']
        analysis = {
            'color_changes': defaultdict(set),
            'shape_changes': set(),
            'symmetry': None,
            'arithmetic': None,
        }
        
        for example in all_examples:
            in_grid = np.array(example['input'])
            out_grid = np.array(example['output'])
            
            analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
            
            # Track color changes
            for (i,j), val in np.ndenumerate(in_grid):
                if i < out_grid.shape[0] and j < out_grid.shape[1]:
                    if in_grid[i,j] != out_grid[i,j]:
                        analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
        
        return analysis
    
    def generate_solution(self, task_data):
        """Generate solution for task"""
        analysis = self.analyze_task(task_data)
        
        for handler in self.pattern_handlers:
            try:
                solution = handler(task_data, analysis)
                if solution and self.verify_solution(solution, task_data):
                    return solution
            except:
                continue
        
        # Default fallback
        return """def p(g):
 return g
"""
    
    def verify_solution(self, solution_code, task_data):
        """Verify solution works on training examples"""
        try:
            exec(solution_code, globals())
            for example in task_data['train'][:3]:
                input_grid = example['input']
                expected = example['output']
                actual = p(input_grid)
                if actual != expected:
                    return False
            return True
        except:
            return False


def create_arc_solutions(input_dir=".", output_dir="submission"):
    """Create solutions for all tasks"""
    generator = ARCSolutionGenerator()
    solutions = {}
    
    os.makedirs(output_dir, exist_ok=True)
    
    solved_count = 0
    total_count = 0
    
    # Process each task
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        task_file = os.path.join(input_dir, f"task{task_id}.json")
        
        try:
            with open(task_file) as f:
                task_data = json.load(f)
            
            solution = generator.generate_solution(task_data)
            solutions[task_id] = solution
            
            if generator.verify_solution(solution, task_data):
                print(f"✅ Task {task_id} solved")
                solved_count += 1
            else:
                print(f"❌ Task {task_id} fallback")
            
            total_count += 1
                
        except Exception as e:
            print(f"Error task {task_id}: {str(e)}")
            solutions[task_id] = """def p(g):
 return g
"""
            total_count += 1
    
    # Write solution files
    for task_id, code in solutions.items():
        output_file = os.path.join(output_dir, f"task{task_id}.py")
        with open(output_file, "w") as f:
            f.write(code)
    
    # Create zip
    with zipfile.ZipFile("submission.zip", "w") as zipf:
        for task_id in solutions:
            file_path = os.path.join(output_dir, f"task{task_id}.py")
            zipf.write(file_path, f"task{task_id}.py")
    
    print(f"\n✅ Solved: {solved_count}/{total_count} ({solved_count/total_count*100:.1f}%)")
    print("📦 Created: submission.zip")

# Run the solution generator
create_arc_solutions(input_dir="/kaggle/input/google-code-golf-2025")